In [1]:
import collections
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [2]:
data = pd.read_json('../data/News_Category_Dataset_v3.json', lines=True)

In [3]:
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [4]:
data.category.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

list_of_headlines = df.loc[df['category']=='CRIME', 'headline'].to_list()

list_of_words = [word for headline in list_of_headlines for word in headline.split()]
words_dict = collections.Counter(list_of_words)
words_dict

df = pd.DataFrame.from_dict(words_dict, orient='index').reset_index().rename(columns={"index": "word", 0: "count"})
df.head()

In [5]:
def punctioation_fix(sentence):  
    tokenizer = RegexpTokenizer(r'\w+')
    words_list = tokenizer.tokenize(sentence.lower())
    return words_list

In [6]:
def words_stemming(words_list):    
    ps = PorterStemmer()
    stem_words = [ps.stem(word) for word in words_list]
    return stem_words

In [7]:
def reduce_stop_words(words_list):
    text = ' '.join(words_list)
    filtered_sentence = remove_stopwords(text)
    return filtered_sentence.split()

In [8]:
def adjust_headline(sentence):    
    words_list = punctioation_fix(sentence)
    
    words_list = [word for word in words_list if len(word) > 1]        
    
    words_list = words_stemming(words_list)
    
    words_list = reduce_stop_words(words_list)
    
    return ' '.join(words_list)

In [9]:
def create_vocabulary(data, label_column, data_column):
    list_of_headlines = data[data_column].apply(adjust_headline).tolist()
    set_of_words = set([word for headline in list_of_headlines for word in headline.split()])

    columns = data[label_column].unique().tolist()
    columns =  [col.lower().replace('&', 'n').replace(' ', '_') for col in columns]

    columns_dict = {i: col for i, col in enumerate(columns)}
    columns_dict['index'] = 'word'
    df = pd.DataFrame(index=set_of_words , columns=np.arange(len(columns))).reset_index().rename(columns=columns_dict).fillna(0)
    
    return df

words_vector_df = create_vocabulary(data, 'category', 'headline')

In [10]:
words_vector_df.head()

,word,u.s._news,comedy,parenting,world_news,culture_n_arts,tech,sports,entertainment,politics,...,style,green,taste,healthy_living,the_worldpost,good_news,worldpost,fifty,arts,divorce
0,idaho,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,cring,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,publicli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,flake,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,righti,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


for cat in data['category'].unique()[:1]:
    print(f'{cat=}')
    headlines = data.loc[data['category']==cat, 'headline'].to_list()
    print(f'{headlines=}')
    words_list = [word for headline in headlines for word in adjust_headline(headline).split()]
    print(f'{words_list=}')
    words_dict = collections.Counter(words_list)
    print(f'{words_dict=}')
          
    for word, value in words_dict.items():
        df.loc[df["word"] == word, cat] = value

In [11]:
def update_category_values(df, data):
    for cat in data['category'].unique():
        headlines = data.loc[data['category']==cat, 'headline'].to_list()
        words_list = [word for headline in headlines for word in adjust_headline(headline).split()]
        words_dict = collections.Counter(words_list)
        
        adjusted_cat = cat.lower().replace('&', 'n').replace(' ', '_')
        for word, value in words_dict.items():
            df.loc[df["word"] == word, adjusted_cat] = value
            
    return df
words_vector_df = update_category_values(words_vector_df, data)

In [12]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


words_vector_df.head(50)

,word,u.s._news,comedy,parenting,world_news,culture_n_arts,tech,sports,entertainment,politics,weird_news,environment,education,crime,science,wellness,business,style_n_beauty,food_n_drink,media,queer_voices,home_n_living,women,black_voices,travel,money,religion,latino_voices,impact,weddings,college,parents,arts_n_culture,style,green,taste,healthy_living,the_worldpost,good_news,worldpost,fifty,arts,divorce
0,idaho,0,0,0,0,0,1,0,1,15,2,0,0,6,0,1,0,0,0,0,3,1,0,0,4,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,cring,0,2,2,0,0,0,1,4,3,0,0,0,1,1,0,0,5,1,0,1,0,0,0,1,0,0,0,0,2,0,0,0,1,0,1,1,0,0,0,0,0,0
2,publicli,0,0,3,0,0,2,1,4,13,1,1,2,0,0,0,1,0,0,0,8,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
3,flake,0,1,0,0,0,0,0,0,26,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,righti,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,hygg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,gevo,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,necrophiliac,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,visag,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,free,2,4,39,12,0,15,26,45,149,12,17,4,10,2,129,34,31,70,15,15,76,10,17,87,32,15,4,44,20,14,20,10,11,17,36,37,22,28,12,3,8,11


words_vector_df = create_vocabulary(data, 'category', 'headline')
words_vector_df = update_category_values(words_vector_df, data)